## Prompt

You are an Application Security analyst focused on identifying security-relevant issues in pull request discussions. Scan the PR metadata and discussion for mentions that align with the following problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has a "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide a summary of the identified issues in JSON format. Each issue must be an object containing the keys "category", which is the category cited with greatest relation with the problem, and "issue", which is a concise summary of the issue itself. Report issues ONLY if there are explicit security terms, concerns, or code changes related to security, without inferences. If no explicit security considerations have been mentioned in the pull request discussion, return exactly []. Do not add any preambles."

In [1]:
import requests
import json
import pprint
import re

In [2]:
def prompt_formatting(system, user):
    return [
        {"role": "system", "content": system},
        {"role": "user", "content": user}
    ]

In [6]:
url = "http://localhost:11434/api/generate"
headers = {
    "Content-Type": "application/json",
}

def llm(model, prompt):
    data_llm = {
        "model": model,
        "prompt": str(prompt),
        "stream": False,
        "options": {
            "temperature": 0.0
        }

    }

    response = requests.post(url, headers=headers, data=json.dumps(data_llm))
    if response.status_code == 200:
        response_data = response.json()
        text = response_data.get('response', '').strip()
        return text

In [63]:
system_prompt = """You are an Application Security analyst focused on identifying security-relevant issues in pull request discussions. Scan the PR metadata and discussion for mentions that align with the following problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has a "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide a summary of the identified issues in JSON format. Each issue must be an object containing the keys "category", which is the category cited with greatest relation with the problem, and "issue", which is a concise summary of the issue itself. Report issues ONLY if there are explicit security terms, concerns, or code changes related to security, without inferences. If no explicit security considerations have been mentioned in the pull request discussion, return exactly []. Do not add any preambles."""

In [64]:
user_prompt = {
  "pr": {
    "title": "Add new black mode funcionality",
    "description": "Implements black mode function on the dashboard\n"
  },
  "threads": [
    {
      "scope": "PR",
      "discussion": [
        "The implementation looks good, but add tests for the new feature.",
        "Tests have been added in the latest commit."
      ]
    }
  ]
}

In [65]:
response = llm("mistral:7b", prompt_formatting(system_prompt, user_prompt))
print(response)

[{"category": "Security Misconfiguration", "issue": "Lack of security testing for the newly implemented black mode functionality"}]


In [66]:
user_prompt = {
  "pr": {
    "title": "Add new black mode funcionality",
    "description": "Implements black mode function on the dashboard\n"
  },
  "threads": [
    {
      "scope": "PR",
      "discussion": [
        "The implementation looks good!",
      ]
    }
  ]
}

In [67]:
response = llm("mistral:7b", prompt_formatting(system_prompt, user_prompt))
print(response)

[]


In [68]:
user_prompt = {
  "pr": {
    "title": "Add customer analytics export feature",
    "description": "Introduces an export endpoint and a CLI to generate customer analytics reports.\n"
  },
  "threads": [
    {
      "scope": "PR",
      "discussion": [
        "Security issue: hostile input is concatenated into SQL and shell commands. Please replace dynamic string-built SQL with parameterized queries/whitelisted identifiers, and switch shell execution to spawn/execFile with validated args (no shell interpolation). Add tests for injection attempts.",
        "Fixed: SQL now uses placeholders with a whitelist for sort fields; CLI moved to spawn with args and strict path/flag validation. Added negative tests for SQL/command injection."
      ]
    },
    {
      "scope": "FILE:server/db/reports.ts",
      "discussion": [
        "The query concatenates req.query.status, req.query.since, and req.query.sort into the SQL string. This is vulnerable to injection. Use placeholders ($1, $2, …) for values and a strict allowlist for sortable columns/directions instead of interpolating them.",
        "Updated: values bound via parameters; sort field/direction validated against an allowlist before constructing ORDER BY. No user data is concatenated into SQL."
      ]
    }
  ]
}

In [69]:
response = llm("mistral:7b", prompt_formatting(system_prompt, user_prompt))
print(response)

[
  {
    "category": "Injection",
    "issue": "The pull request introduces a vulnerability to SQL injection due to dynamic string-building in the SQL query. The issue has been addressed by replacing dynamic string-built SQL with parameterized queries and whitelisting identifiers."
  },
  {
    "category": "Injection",
    "issue": "The pull request also introduces a vulnerability to command injection through shell execution. The issue has been resolved by switching shell execution to spawn/execFile with validated arguments and strict path/flag validation."
  }
]


In [70]:
user_prompt = {
  "pr": {
    "title": "Add tenant setup wizard",
    "description": "Implements initial admin creation and onboarding flow for new tenants.\n"
  },
  "threads": [
    {
      "scope": "FILE:infrastructure/sql/init.sql",
      "discussion": [
        "This seeds a default admin with password 'admin'. Do not create credentials in production images. Remove this and create no user by default.",
        "Done: dropped admin seeding; migrations now create tenant metadata only. Admin is created via setup link without a default password."
      ]
    },
    {
      "scope": "FILE:server/auth/passwordPolicy.ts",
      "discussion": [
        "Current validator permits 'Password1' and similar. Enforce: length >= 12, deny common/well-known passwords, forbid username/email substrings, and require character variety or entropy threshold. Apply in createUser and resetPassword.",
        "Implemented policy and wired into create/reset paths; added tests that reject 'Password1', 'adminadmin', and known-common passwords."
      ]
    }
  ]
}

In [71]:
response = llm("mistral:7b", prompt_formatting(system_prompt, user_prompt))
print(response)

[
    {
      "category": "Broken Access Control",
      "issue": "The initial implementation of the admin creation and onboarding flow for new tenants does not seem to have any access control checks, as a default admin is being seeded without password in production images."
    },
    {
      "category": "Identification and Authentication",
      "issue": "The current password policy in 'server/auth/passwordPolicy.ts' allows weak passwords such as 'Password1', which should be enforced to meet stronger security standards (length >= 12, deny common/well-known passwords, forbid username/email substrings, and require character variety or entropy threshold)."
    }
  ]


In [72]:
# Exemplo 1: Caso sem segurança (deve retornar [])
user_prompt_no_security = {
  "pr": {
    "title": "Update CSS styles for better responsiveness",
    "description": "Improves mobile layout and adds dark theme support"
  },
  "threads": [
    {
      "scope": "PR", 
      "discussion": [
        "The new styles look great! Mobile experience is much better now.",
        "Thanks! I also added some animations for smoother transitions."
      ]
    },
    {
      "scope": "FILE:styles/main.css",
      "discussion": [
        "Consider using CSS variables for the color scheme to make theming easier.",
        "Good idea, I'll refactor that in the next commit."
      ]
    }
  ]
}

# Exemplo 2: Broken Access Control (A01)
user_prompt_access_control = {
  "pr": {
    "title": "Add admin panel endpoints",
    "description": "Creates new admin endpoints for user management"
  },
  "threads": [
    {
      "scope": "FILE:routes/admin.py",
      "discussion": [
        "These admin endpoints are missing authorization checks. Any authenticated user can access admin functions.",
        "You're right, I need to add role-based access control to restrict these to admin users only.",
        "Added @require_admin decorator to all admin endpoints."
      ]
    }
  ]
}

# Exemplo 3: Cryptographic Failures (A02)
user_prompt_crypto = {
  "pr": {
    "title": "Implement user password reset",
    "description": "Adds password reset functionality via email tokens"
  },
  "threads": [
    {
      "scope": "FILE:auth/reset.py",
      "discussion": [
        "The reset tokens are using MD5 hashing which is cryptographically broken. Please use a secure hash like SHA-256 or bcrypt.",
        "Also, the tokens don't have expiration time which is a security risk.",
        "Updated to use SHA-256 with salt and added 1-hour expiration for reset tokens."
      ]
    }
  ]
}

# Exemplo 4: Injection vulnerabilities (A03) 
user_prompt_injection = {
  "pr": {
    "title": "Add search functionality",
    "description": "Implements search across user profiles and posts"
  },
  "threads": [
    {
      "scope": "FILE:search/query.py",
      "discussion": [
        "This search function is vulnerable to SQL injection. The user input is directly concatenated into the SQL query.",
        "Please use parameterized queries or an ORM to prevent SQL injection attacks.",
        "Fixed: replaced string concatenation with parameterized queries using ? placeholders."
      ]
    }
  ]
}

# Exemplo 5: Security Misconfiguration (A05)
user_prompt_misconfig = {
  "pr": {
    "title": "Deploy application to production",
    "description": "Adds Docker configuration and deployment scripts"
  },
  "threads": [
    {
      "scope": "FILE:Dockerfile",
      "discussion": [
        "The application is running as root user in the container, which poses security risks.",
        "Also, debug mode is still enabled in the production config.",
        "Created non-root user for the container and disabled debug mode in production."
      ]
    }
  ]
}

# Exemplo 6: Vulnerable Components (A06)
user_prompt_components = {
  "pr": {
    "title": "Update dependencies",
    "description": "Updates various npm packages to latest versions"
  },
  "threads": [
    {
      "scope": "FILE:package.json",
      "discussion": [
        "Several dependencies have known security vulnerabilities. The lodash version has CVE-2020-8203.",
        "Express.js version also has security patches available.",
        "Updated all dependencies to latest secure versions and ran security audit."
      ]
    }
  ]
}

# Exemplo 7: Identification and Authentication (A07)
user_prompt_auth = {
  "pr": {
    "title": "Implement user authentication",
    "description": "Adds login and registration functionality"
  },
  "threads": [
    {
      "scope": "FILE:auth/login.py",
      "discussion": [
        "The login system has no brute force protection. Attackers can try unlimited password attempts.",
        "Password requirements are too weak - allowing passwords like '123456'.",
        "Added rate limiting after 5 failed attempts and enforced strong password policy (min 8 chars, mixed case, numbers)."
      ]
    }
  ]
}

# Exemplo 8: Security Logging (A09)
user_prompt_logging = {
  "pr": {
    "title": "Add payment processing",
    "description": "Implements credit card payment handling"
  },
  "threads": [
    {
      "scope": "FILE:payments/processor.py",
      "discussion": [
        "Critical security events like failed payments and fraud attempts are not being logged.",
        "We need audit trails for compliance and security monitoring.",
        "Added comprehensive logging for all payment events, failed attempts, and suspicious activities."
      ]
    }
  ]
}

# Exemplo 9: Server-Side Request Forgery (A10)
user_prompt_ssrf = {
  "pr": {
    "title": "Add webhook functionality",
    "description": "Allows users to configure webhooks for notifications"
  },
  "threads": [
    {
      "scope": "FILE:webhooks/sender.py",
      "discussion": [
        "The webhook URL validation is insufficient. Users can provide internal network URLs which could lead to SSRF attacks.",
        "Need to whitelist allowed domains or block internal IP ranges.",
        "Added URL validation to block internal IPs (192.168.x.x, 10.x.x.x, 127.x.x.x) and restrict to HTTPS only."
      ]
    }
  ]
}

# Exemplo 10: Caso ambíguo (menciona segurança mas não é explícito)
user_prompt_ambiguous = {
  "pr": {
    "title": "Refactor user service",
    "description": "Improves user data handling and performance"
  },
  "threads": [
    {
      "scope": "FILE:services/user.py",
      "discussion": [
        "The refactoring looks good, but make sure we're not exposing sensitive data.",
        "Performance improvements are significant, great work!",
        "I've been careful about data exposure, all sensitive fields are properly filtered."
      ]
    }
  ]
}

# Exemplo 11: Múltiplas categorias de segurança
user_prompt_multiple = {
  "pr": {
    "title": "Security hardening for user management",
    "description": "Multiple security improvements for user handling"
  },
  "threads": [
    {
      "scope": "FILE:auth/middleware.py",
      "discussion": [
        "Found several security issues: 1) No rate limiting on login attempts 2) Passwords stored in plain text 3) No input sanitization",
        "These are critical vulnerabilities that need immediate fixing.",
        "Fixed: Added bcrypt password hashing, implemented rate limiting, and added input validation/sanitization."
      ]
    },
    {
      "scope": "FILE:models/user.py", 
      "discussion": [
        "User queries are vulnerable to SQL injection attacks.",
        "Replaced raw SQL with ORM queries to prevent injection."
      ]
    }
  ]
}

In [73]:
# Teste cada exemplo
examples = [
    ("No Security", user_prompt_no_security),
    ("Access Control", user_prompt_access_control), 
    ("Cryptographic", user_prompt_crypto),
    ("Injection", user_prompt_injection),
    ("Misconfiguration", user_prompt_misconfig),
    ("Vulnerable Components", user_prompt_components),
    ("Authentication", user_prompt_auth),
    ("Security Logging", user_prompt_logging),
    ("SSRF", user_prompt_ssrf),
    ("Ambiguous", user_prompt_ambiguous),
    ("Multiple Issues", user_prompt_multiple)
]

for name, prompt in examples:
    print(f"\n=== Testing {name} ===")
    response = llm("mistral:7b", prompt_formatting(system_prompt, prompt))
    print(response)
    print("-" * 50)


=== Testing No Security ===
[{"category": "Insecure Design", "issue": "Lack of proper input validation and sanitization on user-supplied data in CSS variables could lead to Injection attacks."}]
--------------------------------------------------

=== Testing Access Control ===
[{"category": "Broken Access Control", "issue": "The pull request introduces new admin endpoints without initial authorization checks, allowing any authenticated user to access admin functions. The issue has been addressed by adding a @require_decorator for admin users."}]
--------------------------------------------------

=== Testing Cryptographic ===
[{"category": "Cryptographic Failures", "issue": "The initial implementation of password reset tokens used MD5 hashing, which is cryptographically broken. The issue has been resolved by updating to use SHA-256 with salt."}, {"category": "Security Misconfiguration", "issue": "Initially, the password reset tokens did not have an expiration time, creating a security